# 요약

- 개요
    - 주식의 가격에는 다양한 요소들이 영향을 미치지만 그 중 뉴스에 민감하다고 판단하여 뉴스 기사의 제목을 분석하여 긍정/부정 평가를 한다.
1. 데이터 수집
    - 최근 1일, 정확도 순서로 검색어 입력 시 그에 대한 뉴스 제목 정보를 크롤링한다.
2. 전처리
    - konlpy의 Hannanum으로 제목에 대해서 형태소 분석을 하고 tokenize를 진행한다.
3. 모델링
    - pre-train 된 한국식 bert 모델인 koBart 모델을 사용하여 sentiment(감성) 점수를 부여한다.
    - 제목을 수치화한 'sentiment' 값을 얻어내고 0.5 기준으로 이상이면 긍정, 미만이면 부정으로 'label' 이라고 labeling을 수행
4. 결과
    - 두 가지 방식으로 긍/부정을 결론 짓는다.
        - 모든 뉴스의 감성 점수에 대한 평균을 기준으로 0.5 미만이면 부정/ 0.5 이상이면 긍정
        - 각각의 뉴스에 대해 미리 긍/부정을 나눈 'label'의 개수에 대해 긍정 뉴스가 많으면 긍정, 부정 뉴스가 많으면 부정으로 결론짓는다.

# 네이버 뉴스 크롤링
### 1. 데이터 수집
- 네이버 검색창에 키워드에 대한 뉴스를 크롤링함
- 정확도순으로 최근 1일의 뉴스 데이터를 100개 이하로 수집
- requests 라이브러리로 url에 접근하고 BeautifulSoup으로 뉴스 기사 본문을 파싱하지만 기사 제목 수준으로만 감성 분성을 진행함

In [1]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import pandas as pd

# 페이지 입력 (1 페이지당 기사 10개 이하)
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# search : 검색어, pd=4 : 최근 1일, start_page : 몇 페이지
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&start=" + str(
            start_page)
        
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            #url = "https://search.naver.com/search.naver?where=news&query=%EC%B9%B4%EC%B9%B4%EC%98%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.09.10&de=2021.09.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210910to20210915&is_sug_officeid=0"+"&start=" + str(page)
            urls.append(url)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)

# 기사 내용 크롤링 함수
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url



#### 입력부

In [2]:
#####뉴스크롤링 시작#####

# 검색어 입력
search = input("검색 키워드 입력 : ")
# 검색 시작할 페이지 입력
page = 1
# 검색 종료할 페이지 입력
page2 = 10

searches = ['삼성전자','LG에너지솔루션','SK하이닉스','삼성바이오로직스','POSCO홀딩스',"LG화학","삼성SDI","현대차","NAVER"]

# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []

for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
# ------------------------------------------------------------------------------------------------------------------
    # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')
    
    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
        news_date = news_date[0:10]
    # 날짜 가져오기
    news_dates.append(news_date)

print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print("\n[뉴스 날짜]")
print(news_dates)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

검색 키워드 입력 : sk하이닉스


100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:08<00:00,  7.15it/s]


[뉴스 제목]
["'FMS 2023' 개막…삼성·SK하이닉스, AI·車 반도체 대거 공개", '"水·電 문제에 TSMC 2나노 공장 첫삽도 못 떠"...삼성전자·SK하이닉스 \'동병상련\'', '[부고]김동섭(SK하이닉스 사장)씨 빙부상', '[부고]\xa0김동섭(SK하이닉스 사장)씨 장인상 외', '[부고]\xa0김동섭\xa0SK하이닉스 사장 장인상', '[단독] 한미반도체, 엔비디아향 12단 HBM3 TC본더 독점 공급', "한반도 북상 태풍 '카눈'…삼성·SK, 반도체 공장도 '긴장'", 'SK그룹이 어쩐 일로…그룹주 승자됐네 [분석+]', '코스피, 하락 전환해 2600선 하회…반도체·인터넷株↓', 'HBM 올라탄 한미반도체, 생산능력·고객사 확대한다', '[단독]반도체장비 유입, 韓-中만 줄었다… 공급망 재편 타격', '삼성·LGD, 올레도스·마이크로LED 등 ‘초격차’ 기술투자에 속도낸다', '넉달 째 뛴 글로벌 반도체 매출… 문제는 낸드 회복세', "꼬인 수급 풀리니 실적株로…'에코프로 내리고 네이버 오르고'", "[어제장 오늘장] '청구서' 받아 든 2차전지…쏠림 풀리자 '와르르'", '반도체주 오르는데 꿈쩍않는 삼성전자… 주가 한달째 지지부진', '반도체 불황 터널, 끝이 보인다', '반도체 불황에도 성과급 쏘는 삼성·SK, 왜? [재계 TALK TALK]', '기업들, 잼버리 참가자 대상 견학 프로그램 잇따라 내놓아', '이차전지 흔들리자 증시 출렁… 반도체로 대이동', '[오전 시황] 코스피, 닷새 만에 오름세 돌아서…2600 탈환 시도', "'6만전자' 개미 속 타는데…'9만전자' 외치는 증권가", '“반도체팹 만전 기하라”...‘카눈’ 북상에 삼성·SK 비상 체제', "코스피, 美 훈풍에 장 초반 상승…개인 '사자'[개장시황]", "AI 열풍에 게임용 GPU까지 품귀…'메모리 반등' 앞당길까", '[코스피 마감]닷새 연속 하락해 2570선…초전도체株 폭락', '잼버리 지원 나선 현대차…전주공장 견학 프로그램도', '낸드플래시 시장에 ‘

In [3]:
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'content' : news_contents})
news_df

,date,title,content
0,2023-08-08 11:50:33,"'FMS 2023' 개막…삼성·SK하이닉스, AI·車 반도체 대거 공개",[]
1,2023-08-08 05:01:07,"""水·電 문제에 TSMC 2나노 공장 첫삽도 못 떠""...삼성전자·SK하이닉스 '동...",[]
2,2023-08-07 18:38:11,[부고]김동섭(SK하이닉스 사장)씨 빙부상,[]
3,2023-08-07 17:14:01,[부고] 김동섭(SK하이닉스 사장)씨 장인상 외,[]
4,2023-08-07 17:12:03,[부고] 김동섭 SK하이닉스 사장 장인상,[]
5,2023-08-08 08:51:01,"[단독] 한미반도체, 엔비디아향 12단 HBM3 TC본더 독점 공급",[]
6,2023-08-08 06:31:07,"한반도 북상 태풍 '카눈'…삼성·SK, 반도체 공장도 '긴장'",[]
7,2023-08-08 08:37:01,SK그룹이 어쩐 일로…그룹주 승자됐네 [분석+],[]
8,2023-08-08 13:52:01,"코스피, 하락 전환해 2600선 하회…반도체·인터넷株↓",[]
9,2023-08-07 17:06:01,"HBM 올라탄 한미반도체, 생산능력·고객사 확대한다",[]


### 2. 데이터 전처리 및 모델 적용
- 제목을 단어 단위로 토큰화를 수행함.
    - konlpy의 Hannanum을 선택함.
- 토큰화된 제목을 koBart 모델을 사용하여 감성 수치를 추출하여 'sentiment'로 저장한다.
- sentiment 값을 토대로 0.5 이상이면 1, 0.5 미만이면 0으로 하여 'label'로 저장한다.

In [4]:
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

df = news_df

# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# title 열에 대해 형태소 분석 적용

df['title'] = df['title'].astype(str).apply(tokenize)
# for i in range(df.shape[0]):
#     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.dense.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
df

,date,title,content,sentiment,label
0,2023-08-08 11:50:33,"[', FMS, 2023, ', 개막…삼성·SK하이닉스,, AI, ·, 車, 반도체...",[],0.31,0
1,2023-08-08 05:01:07,"[""水·電, 문제, 에, TSMC, 2나노, 공장, 첫삽, 도, 못, 뜨, 어, ""...",[],0.53,1
2,2023-08-07 18:38:11,"[[부고]김동섭(SK하이닉스, 사장, ), 씨, 빙부상]",[],0.60,1
3,2023-08-07 17:14:01,"[[부고] 김동섭(SK하이닉스, 사장, ), 씨, 장인상, 외]",[],0.53,1
4,2023-08-07 17:12:03,"[[부고] 김동섭 SK하이닉스, 사장, 장인상]",[],0.59,1
5,2023-08-08 08:51:01,"[[, 단독, ], 한미반도체,, 엔비디아향, 12단, HBM3, TC본더, 독점,...",[],0.54,1
6,2023-08-08 06:31:07,"[한반, 도, 북상, 태풍, '카눈'…삼성·SK,, 반도체, 공장, 도, ', 긴장...",[],0.52,1
7,2023-08-08 08:37:01,"[SK, 그룹, 이, 어쩌, ㄴ, 일로…그룹주, 승자, 되, 었네, [분석+, ]]",[],0.50,0
8,2023-08-08 13:52:01,"[코스피, ,, 하락, 전환해, 2600, 선, 하회…반도체·인터넷株↓]",[],0.51,1
9,2023-08-07 17:06:01,"[HBM, 오르, 아, 타, ㄴ, 한미반도체,, 생산능력·고객사, 확대, 하, ㄴ다]",[],0.52,1


### 3. 결과
#### 1) 평균으로 분석
- 뉴스 기사의 감성 점수의 평균이 0.5 이상이면 다음 날의 주가의 상승을 예측
- 반대의 경우 하락으로 예측함

In [6]:
print(f"모든 뉴스에 대한 평균 감성 점수 값: {df['sentiment'].mean()}")
if df['sentiment'].mean() >= 0.5:
    print("주가의 상승 예측")
else:
    print("주가의 하락 예측")

모든 뉴스에 대한 평균 감성 점수 값: 0.538
주가의 상승 예측


#### 2) 개수로 분석
- label의 값이 1인 경우의 뉴스가 더 많은 경우에 다음날의 주가의 상승을 예측
- 반대의 경우 하락을 예측함.

In [7]:
sentiment = 0
print(f"긍정 뉴스의 개수: {df[df['label'] == 1].label.count()}")
print(f"부정 뉴스의 개수: {df[df['label'] == 0].label.count()}")

if df[df['label'] == 1].label.count() > df[df['label'] == 0].label.count():
    sentiment = 1
    print("주가 상승으로 예측")
else:
    sentiment = 0
    print("주가 하락으로 예측")



긍정 뉴스의 개수: 44
부정 뉴스의 개수: 16
주가 상승으로 예측


### csv로 만들면서 마무리

In [33]:
###데이터 프레임으로 만들기###

# 데이터 프레임 만들기
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 데이터 프레임 저장
now = datetime.datetime.now()
date = str(now.year%100)+format(now.month,'02')+format(now.day,'02')
df.to_csv(f'csv/{date}_{search}_감성분석.csv', encoding='utf-8-sig', index=False)